In [4]:
import re

from pathlib import Path
import matplotlib.pyplot as plt

In [10]:
result_dir = Path().parent / 'result'
num_particle_result_dir = result_dir / 'num_particles'
num_layer_result_dir = result_dir / 'num_layer'

result_dict = {
    'seq': {
        'num_particles': {},
        'num_layer': {},
    },
    'omp': {
        'num_particles': {},
        'num_layer': {},
    },
    'mpi': {
        'num_particles': {},
        'num_layer': {},
    },
    'hip': {
        'num_particles': {},
        'num_layer': {},
    },
}

time_regex = re.compile('Time: ([0-9]*[.]?[0-9]+)')

In [ ]:
for prefix in result_dict
    for i in range(1, 7):
        particle_result_file = num_particle_result_dir / f'{prefix}_a30k_w{i}_time.txt'
        running_time = []
        with particle_result_file.open() as f:
            for line in f:
                match = time_regex.match(line)
                if match:
                    running_time.append(float(match.group(1)))
        assert len(running_time) == 10
        average_running_time = sum(running_time)/len(running_time)
        result_dict[prefix]['num_particles'][i*20000] = average_running_time
    
    num_layer = 1
    for i in range(1, 21):
        layer_result_file = num_layer_result_dir / f'{prefix}_a{num_layer}_w1_time.txt'
        running_time = []
        with layer_result_file.open() as f:
            for line in f:
                match = time_regex.match(line)
                if match:
                    running_time.append(float(match.group(1)))
        assert len(running_time) == 10
        average_running_time = sum(running_time)/len(running_time)
        result_dict[prefix]['num_layer'][num_layer] = average_running_time
        num_layer *= 2

In [ ]:
num_particles_list = None
for prefix in result_dict:
    if num_particles_list is None:
        num_particles_list = sorted(list(result_dict[prefix]['num_particles'].keys()))
        
    running_times = []
    for num_particle in num_particles_list:
        running_times.append(result_dict[prefix]['num_particles'][num_particle])
    
    plt.plot(num_particles_list, running_times, label=prefix)
    
plt.xlabel("Number of particles")
plt.ylabel("Running time")
plt.legend()
plt.title("Running time as number of particles scales")
plt.show()